In [ ]:
import numpy as np
import os, re, csv, math, codecs
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer as t
from tqdm import tqdm
from keras.layers import merge, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import *
# to visualize, and to make zero shape matrix
from attention_utils import get_activations, get_data_recurrent
from Attention import Attention
import pandas as pd
from konlpy.tag import Okt as Twitter
from selfword2vec import tokenization
from Anomaly import checkAnomaly_x_y
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from numpy import argmax
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import fbeta_score
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.losses import binary_crossentropy
import json
# 1. 모델 저장시키기 
# 2. tokenizer와 konlpy morphs 호환 여부 (완성 jupyter note)
# 2'. word2vec 2가지 경우 더 추가 to embedding(final_total word2vec, twitter_translated.vec)
# 3. 변수들 설정하기


np.random.seed(3)

# NUM_WORDS, train으로 input받은 단어의 수
MAX_NB_WORDS = 20000
vocab_size = 0
EMB_DIM = 300
embeddings_index = dict()

# columns = ["ID", "Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism","sadness","surprise","trust"]
columns = ["ID","Tweet","분노","기대","혐오스러운","두려움","기쁨","사랑","낙관론","비관론","슬픔","놀라움","믿음"]




##########################################1. 데이터셋 생성하기

train_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_train.txt",sep="\t", header=None,names=columns).values
val_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_dev.txt",sep="\t", header=None,names=columns).values
test_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_test_gold.txt",sep="\t", header=None,names=columns).values

# 판다 shape
print("train_array"+ str(train_array.shape))
print("val_array"+str(val_array.shape))
print("test_array"+str(test_array.shape))

print("Reading data!")
#  x, y 분할하기
x_train = train_array[1:,1]
y_train = train_array[1:,2:]
x_val = val_array[1:,1]
y_val = val_array[1:,2:]
x_test = test_array[1:,1]
y_test = test_array[1:,2:]


print("checking Anomaly!!")
x_train, y_train = checkAnomaly_x_y(x_train,y_train)
x_val, y_val = checkAnomaly_x_y(x_val,y_val)
x_test, y_test = checkAnomaly_x_y(x_test,y_test)

print("ANomaly result!")
print("x_train.shape"+ str(x_train.shape))
print("y_train.shape"+ str(y_train.shape))
print("x_val.shape"+ str(x_val.shape))
print("y_val.shape"+ str(y_val.shape))
print("x_test.shape"+ str(x_test.shape))
print("y_test.shape"+ str(y_test.shape))
# print(type(x_train)) np.array로 변형이 필요한가
print("Finished!")

# jupyter notebook
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------

tmp = []
train_tmp = []
val_tmp = []
test_tmp = []
max_count = 0
set_words = set()

print("Tokenizing!")
# test의 tmp만 따로 받아주고 나머지는 val train test모두 통합시켜준다.
test_tmp, dummy , set_words = tokenization(x_test)
val_tmp, dummy , set_words = tokenization(x_val)
train_tmp, max_count , set_words = tokenization(x_train)
tmp, dummy , set_words = tokenization(np.hstack([x_train,x_val,x_test]))
print("Tokenizing finished!")
tmp = [] # tmp없애준다



#token shape
print("It's by len()")
print("train_tmp.len():" + str(len(train_tmp))+","+ str(len(train_tmp[0])))
print("val_tmp shape.len():" + str(len(val_tmp))+","+ str(len(val_tmp[0])))
print("test_tmp shape.len():" + str(len(test_tmp))+","+ str(len(test_tmp[0])))


print("max_count:"+str(max_count))
# 문장길이 100으로 맞춘다.
max_count = min(100, max_count)


print("Readding Embedding file")
# embeddings_index == dict(w2v's word: vector)
f = codecs.open('/home/minwookje/coding/ex1_w2v/embedding/1542954106final_total_pos.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


# word2index table 생성 {token:index}
# 모든 train val test에 사용되는 token별 index table
# train 문장 token 갯수
word_index = len(set_words)
word_tmp_dict = dict()
for i, word in enumerate(set_words):
    word_tmp_dict[str(word).replace(" ", "")] = i
word_tmp_dict['0'] = word_index 

#dict file 저장
with open('token2index.json','w') as dictionary_file:
    json.dump(word_tmp_dict,dictionary_file)

## 문장별 토큰화시킨 녀석에 index를 집어 넣어준다. 이때 pad도 동시에 해준다. 
word_vec = []
word_vec_test = []
word_vec_val = []

for sent in train_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break    
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec.append(sub)
## 테스트용 복사본 
for sent in test_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_test.append(sub)

## 검증용 복사본 
for sent in val_tmp:
    sub = []
    for word in sent:
        if(len(sub)==max_count):
            break
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    # padding
    sub.extend([word_index]*count)
    word_vec_val.append(sub)

print("word_vec shape:" + str(len(word_vec))+","+ str(len(word_vec[0])))
print("word_vec_val shape:" + str(len(word_vec_val))+","+ str(len(word_vec_val[0])))
print("word_vec_test shape:" + str(len(word_vec_test))+","+ str(len(word_vec_test[0])))




# 4번쨰 matrix embedding_matrix {index: vector}
# vocab_size = min(MAX_NB_WORDS, word_index)
vocab_size = word_index
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, EMB_DIM))
match_count = 0
unmatch_count = 0

for word, i in word_tmp_dict.items():
    if word != '0':
        if (word in embeddings_index):
            match_count += 1
            # embedding_matrix[i] = np.zeros(300)
            embedding_matrix[i] = embeddings_index[word]
            # embedding_matrix[i] = np.random.uniform(-0.25,0.25,300)
        else:
            unmatch_count += 1
            # embedding_matrix[i] = np.zeros(300)
            # embedding_matrix[i] = np.random.uniform(-0.25,0.25,300) ## used for OOV words
            embedding_matrix[i] = np.random.uniform(-1.0,1.0,300).astype('float32')
print("match:" + str(match_count))
print("unmatch:" + str(unmatch_count))

embedding_matrix.tofile('index2vec.dat')
#     여기부터
# 1.앞서 단어당 벡터 테이블(v) // embeddings_index, {w2v_word: vector}
# train_word(str(word).replace(" ", "")) == embedding
# 2.train 단어별 index (v) // word_tmp_dict {train_word(str(word).replace(" ", "")):index}
# 3.sentence padding, sentence to index
# 4.index당 vector table (v) //embedding_matrix {index: vector} 이녀석을 embedding weight에 넣어주어야 한다. 

# 문장 = [index들 나열 ] 
# 즉 embedding_matrix로 index를 seq에 넣어준묹장들을 train에 넣어줘야한다. 
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------
# np.array화
print("word_vec shape:" + str(len(word_vec))+","+ str(len(word_vec[0])))
print("word_vec_val shape:" + str(len(word_vec_val))+","+ str(len(word_vec_val[0])))
print("word_vec_test shape:" + str(len(word_vec_test))+","+ str(len(word_vec_test[0])))
# np.savetxt('word_vec1.txt', word_vec[:500], delimiter=" ", fmt="%s") 
# np.savetxt('word_vec_val1.txt', word_vec_val[:500], delimiter=" ", fmt="%s") 
# np.savetxt('word_vec_test1.txt', word_vec_test[:500], delimiter=" ", fmt="%s") 

word_vec = np.array(word_vec)
word_vec_val = np.array(word_vec_val)
word_vec_test = np.array(word_vec_test)
embedding_matrix = np.matrix(embedding_matrix)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

print("word_vec shape:" + str(word_vec.shape))
print("word_vec_val shape:" + str(word_vec_val.shape))
print("word_vec_test shape:" + str(word_vec_test.shape))
print("y_train shape:" + str(y_train.shape))
print("y_val shape:" + str(y_val.shape))
print("y_test shape:" + str(y_test.shape))
print("embedding_matrix shape:" + str(embedding_matrix.shape))

# 확인용
np.savetxt('word_vec.txt', word_vec[:500], delimiter=" ", fmt="%s") 
np.savetxt('word_vec_val.txt', word_vec_val[:500], delimiter=" ", fmt="%s") 
np.savetxt('word_vec_test.txt', word_vec_test[:500], delimiter=" ", fmt="%s") 
np.savetxt('embedding_matrix.txt', embedding_matrix[:600], delimiter=" ", fmt="%s") 
# TODO
# 1.padding 23976이 너무 많이 들어간다. 이거 max_count를 정해주어야 한다. 100까지로 줄이자.
# 2.그 다음에는 matrix 사이즈를 맞춰주어야 한다. 


# def fit_multilabel(model, X_train, X_val, y_train, y_val):
#     y_val = np.array(y_val)
#     y_train = np.array(y_train)

#     predictions = np.zeros(y_val.shape)

#     for i in range(y_val.shape[1]):
#         model.fit(X_train, y_train[:, i])
#         y_p = model.predict(X_val)
#         predictions[:, i] = y_p

#     return predictions
# from sklearn.metrics import jaccard_similarity_score

# def jaccard_distance_loss(y_true, y_pred, smooth=100):
    # """
    # Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
    #         = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    # The jaccard distance loss is usefull for unbalanced datasets. This has been
    # shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    # gradient.
    
    # Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    # @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    # @author: wassname
    # """
    # print("loss = %s, %s"%(y_true,y_pred))
    # y_pred = K.cast_to_floatx(y_pred)
    # y_true = K.cast_to_floatx(y_true)
    # intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    # sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    # jac = (intersection + smooth) / (sum_ - intersection + smooth)
    # print("loss = %s, %s, %s"%(intersection, sum, jac))   
    # np.zeros_like(y_true, dtype = object)
    # np.zeros_like(y_pred, dtype = object)

    # im1 = np.asarray(y_true).astype(np.bool)
    # im2 = np.asarray(y_pred).astype(np.bool)
    # intersection = np.logical_and(im1, im2)
    # union = np.logical_or(im1, im2)
    # # intersection = K.sum(np.absolute(y_true * y_pred), axis=-1)
    # # sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    # # jac = (intersection + smooth) / (sum_ - intersection + smooth)
    # jac = (intersection.sum()+smooth) / (float(union.sum()+smooth))
    # print("loss = %s, %s, %s"%(intersection, sum, jac))
    # print("intersection")
    # print(intersection)
    # # print(K.eval(intersection))
    # print("sum")
    # print(sum)
    # # print(K.eval(sum))
    # print("jac")
    # print(jac)
    # # print(K.eval(jac))
    # print("K.eval(1-jac)*smooth")
    # print((1 - jac) * smooth)
    # print(type((1 - jac) * smooth))
    # print(K.eval(1-jac)*smooth)
    # returnimport tensorflow as tf (1 - jac) * smooth

# def jaccard_distance(y_true, y_pred, smooth=100):
#     intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
#     sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
#     jac = (intersection + smooth) / (sum_ - intersection + smooth)
#     return (1 - jac) * smooth
#####################################2. 모델 구성하기
# input_length : 단어의 수 즉 문장의 길이를 나타냅니다
# 임베딩 레이어의 출력 크기는 샘플 수 * output_dim * input_lenth가 됩니다

INPUT_DIM = 300 #wordvec사이즈
max_count = max_count #문장의 max길이
lstm_shape = 250
rate_drop_lstm = 0.3
rate_drop_dense = 0.3

# k_vec = K.variable(word_vec)
# k_vec_val = K.variable(word_vec_val)
# k_vec_test = K.variable(word_vec_test)
# k_y_train = K.variable(y_train)
# k_y_val = K.variable(y_val)
# k_y_test = K.variable(y_test)
# k_embedding_matrix = K.variable(embedding_matrix)
k_vec = word_vec
k_vec_val = word_vec_val
k_vec_test = word_vec_test
k_y_train = y_train
k_y_val = y_val
k_y_test = y_test
k_embedding_matrix = embedding_matrix

# nan delete
index = np.argwhere(np.isnan(k_y_train))[:,0]
index2 = np.argwhere(np.isnan(k_y_val))[:,0]
index3 = np.argwhere(np.isnan(k_y_test))[:,0]

k_y_train = np.delete(k_y_train,index,0)
k_vec = np.delete(k_vec,index,0)
k_y_val = np.delete(k_y_val,index2,0)
k_vec_val = np.delete(k_vec_val,index2,0)
k_vec_test = np.delete(k_vec_test,index3,0)
k_y_test = np.delete(k_y_test,index3,0)
# is nan check
print(np.any(np.isnan(k_vec)))
print(np.any(np.isnan(k_vec_val)))
print(np.any(np.isnan(k_vec_test)))
print(np.any(np.isnan(k_y_train)))
print(np.any(np.isnan(k_y_val)))
print(np.any(np.isnan(k_y_test)))
print(np.any(np.isnan(k_embedding_matrix)))



def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """

    epsilon = tf.convert_to_tensor(1e-7, dtype='float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac)

# matrix accu
def jaccard_distance_acc(y_true, y_pred):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection) / (sum_ - intersection)
    return jac

# def scaled_binary_cross_entropy(y_true,y_pred):
#     epsilon = tf.convert_to_tensor(1e-7, dtype='float32')
#     r =K.binary_crossentropy(y_true,y_pred)
    
#     return r/(r.max()+epsilon)

def scaled_binary_cross_entropy(y_true,y_pred):
    epsilon = tf.convert_to_tensor(1e-7, dtype='float32')
    loss =binary_crossentropy(y_true,y_pred)
    max_t = K.max(loss)
    return loss/(max_t+epsilon)


def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2
    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)
    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)
    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))



accu = jaccard_distance_acc    


# 32하니까 죽어버린다.
batch_size = 32
l2_reg = 0.0001
activity_l2 = 0.0001
# Define the model
# inp = InputLayer(shape=(max_count,), dtype='float32',sparse=True)
# input layer는 
# inp = InputLayer(input_shape=(max_count,),sparse=True)

# inp = Input(shape=(max_count,))
# 생각해보니까 이거 float 필요없다.
inp = Input(shape=(max_count,))
emb = Embedding(input_dim=vocab_size+1, output_dim=EMB_DIM,
            trainable=False, weights=[embedding_matrix], input_length=max_count)(inp)
# max_features = vocab_size, maxlen=text_max_words, embed_size=EMB_DIM
# emb = Embedding(input_dim=max_features, input_length = maxlen, output_dim=embed_size)(inp)
# embedding dropout = 0.1
# x = Bidirectional(LSTM(lstm_shape, return_sequences=True, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm, W_regularizer=l2(l2_reg)))(x)
# weight, bias, hidden state 수정해보기 , 현재는 kernel만 regularization 시켰다. 

x = SpatialDropout1D(0.1)(emb)



x = Bidirectional(LSTM(lstm_shape, return_sequences=True, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,kernel_regularizer=l2(l2_reg)))(x)
x, attention = Attention()(x,rate_drop_dense,l2_reg)


# BatchNOrmalization 추가
# x = BatchNormalization()(x)

# Dense(11, activation="sigmoid",activity_regularizer=activity_l2(0.0001))
x = Dense(11, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

# # 1
# model.compile(loss='binary_crossentropy',
#             optimizer='adam',
#             metrics=[accu])


# 2

# model.compile(loss=jaccard_distance_loss,
#             optimizer='adam',
#             metrics=[accu])

# 3
# model.compile(loss='binary_crossentropy',
#             optimizer='adam',
#             metrics=['accracy'])

# 4
# model.compile(loss='binary_crossentropy',
#             optimizer='rmsprop',
#             metrics=['acc', accu])

# 5
# model.compile(loss='binary_crossentropy',
#             optimizer=Adam(clipnorm=1, lr=0.001),
#             metrics=[accu, fbeta])

# 6
# model.compile(loss=scaled_binary_cross_entropy,
#             optimizer=Adam(clipnorm=1, lr=0.001),
#             metrics=[accu, fbeta])

# 6' clipnorm=1의 역활이 득이 되는지 아닌지 잘 모르겠다. 평가의 기준이 명확하지 않으니까 
# 내가 맞게 평가 하고 학습하는지 잘 모르겠다. 
# model.compile(loss=scaled_binary_cross_entropy,
#             optimizer=Adam(lr=0.001),
#             metrics=[accu, fbeta])


#7
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=0.001),
            metrics=[accu, fbeta])

attention_model = Model(inputs=inp, outputs=attention) # Model to print out the attention data
model.summary()



# EPOCH 16 적당하다.
model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=16, verbose=1, batch_size=batch_size)


# model.fit(k_vec, k_y_train, validation_data=(k_vec_val,k_y_val), epochs=3, verbose=1, steps_per_epoch=int(6619/batch_size)+1, validation_steps = int(883/batch_size)+1)

# 5. 모델 평가하기
test_score = model.evaluate(k_vec_test, k_y_test, batch_size=batch_size)
print('')
print(str(test_score))



# 6. 모델 저장하기
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
# 가중치 저장하기
model.save_weights('test.h5')


Using TensorFlow backend.


train_array(6651, 13)
val_array(887, 13)
test_array(3239, 13)
Reading data!
checking Anomaly!!
(6619,)
(6619, 11)
(883,)
(883, 11)
(3215,)
(3215, 11)
ANomaly result!
x_train.shape(6619,)
y_train.shape(6619, 11)
x_val.shape(883,)
y_val.shape(883, 11)
x_test.shape(3215,)
y_test.shape(3215, 11)
Finished!
Tokenizing!


828it [00:00, 8273.46it/s]

Tokenizing finished!
It's by len()
train_tmp.len():6619,40
val_tmp shape.len():883,39
test_tmp shape.len():3215,46
max_count:10531
Readding Embedding file


45942it [00:05, 7826.50it/s]


Loaded 45942 word vectors.
word_vec shape:6619,100
word_vec_val shape:883,100
word_vec_test shape:3215,100
match:11300
unmatch:12626
word_vec shape:6619,100
word_vec_val shape:883,100
word_vec_test shape:3215,100
word_vec shape:(6619, 100)
word_vec_val shape:(883, 100)
word_vec_test shape:(3215, 100)
y_train shape:(6619, 11)
y_val shape:(883, 11)
y_test shape:(3215, 11)
embedding_matrix shape:(23927, 300)
False
False
False
False
False
False
False


/home/minwookje/coding/ex1_w2v/Attention.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(500, kernel_initializer="glorot_uniform", activation="tanh", name="tanh_mlp", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  x_a = Dense(repeat_size, kernel_initializer = 'glorot_uniform', activation="tanh",name="tanh_mlp",W_regularizer=W_reg,b_regularizer=b_reg)(inp)
/home/minwookje/coding/ex1_w2v/Attention.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(500, kernel_initializer="glorot_uniform", activation="tanh", name="tanh_mlp2", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  x_a = Dense(repeat_size, kernel_initializer = 'glorot_uniform', activation="tanh", name="tanh_mlp2",W_regularizer=W_reg,b_regularizer=b_reg)(x_a)
/home/minwookje/coding/ex1_w2v/Attention.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="glorot_uniform", activation="linear", name="word-leve

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     7178100     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 100, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 500)     1102000     spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
tanh_mlp (